In [ ]:
!unzip concrete_crack_dataset.zip -d cracks_raw

Streaming output truncated to the last 5000 lines.
  inflating: cracks_raw/negative/15001.jpg  
  inflating: cracks_raw/negative/15002.jpg  
  inflating: cracks_raw/negative/15003.jpg  
  inflating: cracks_raw/negative/15004.jpg  
  inflating: cracks_raw/negative/15005.jpg  
  inflating: cracks_raw/negative/15006.jpg  
  inflating: cracks_raw/negative/15007.jpg  
  inflating: cracks_raw/negative/15008.jpg  
  inflating: cracks_raw/negative/15009.jpg  
  inflating: cracks_raw/negative/15010.jpg  
  inflating: cracks_raw/negative/15011.jpg  
  inflating: cracks_raw/negative/15012.jpg  
  inflating: cracks_raw/negative/15013.jpg  
  inflating: cracks_raw/negative/15014.jpg  
  inflating: cracks_raw/negative/15015.jpg  
  inflating: cracks_raw/negative/15016.jpg  
  inflating: cracks_raw/negative/15017.jpg  
  inflating: cracks_raw/negative/15018.jpg  
  inflating: cracks_raw/negative/15019.jpg  
  inflating: cracks_raw/negative/15020.jpg  
  inflating: cracks_raw/negative/15021.jpg  
  in

In [ ]:
import os, shutil, random, glob, json
from pathlib import Path
# =========================
# 1) DATA PREP
#    Expect RAW_DIR/positive and RAW_DIR/negative
# =========================
# TODO: แก้พาธนี้ให้ชี้ไปยังที่คุณวางรูปภาพ (มี positive/negative อยู่ข้างใน)
RAW_DIR = Path("/content/cracks_raw")  # <--- เปลี่ยนตรงนี้

# จะสร้างชุดข้อมูลใหม่ที่นี่
DATASET_DIR = Path("/content/crack_dataset_cls")
SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)

assert (RAW_DIR/'positive').exists() and (RAW_DIR/'negative').exists(), "ไม่พบโฟลเดอร์ positive/negative ใน RAW_DIR"

# สร้างโครงสร้าง train/val/test / (crack|no_crack)
for split in ['train','val','test']:
    for cls in ['crack','no_crack']:
        (DATASET_DIR/split/cls).mkdir(parents=True, exist_ok=True)

# รวบรวมไฟล์
pos_files = sorted([p for p in (RAW_DIR/'positive').rglob('*') if p.suffix.lower() in ['.jpg','.jpeg','.png','.bmp','.tif','.tiff']])
neg_files = sorted([p for p in (RAW_DIR/'negative').rglob('*') if p.suffix.lower() in ['.jpg','.jpeg','.png','.bmp','.tif','.tiff']])

# def split_list(files, train_ratio=0.7, val_ratio=0.15):
#     n = len(files)
#     idx = np.arange(n)
#     np.random.shuffle(idx)
#     n_train = int(n*train_ratio)
#     n_val = int(n*val_ratio)
#     train_idx = idx[:n_train]
#     val_idx = idx[n_train:n_train+n_val]
#     test_idx = idx[n_train+n_val:]
#     return [files[i] for i in train_idx], [files[i] for i in val_idx], [files[i] for i in test_idx]

def split_list(files, train_ratio=0.7, val_ratio=0.15, seed=42):
    files = list(files)
    rnd = random.Random(seed)
    rnd.shuffle(files)
    n = len(files)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    train = files[:n_train]
    val = files[n_train:n_train + n_val]
    test = files[n_train + n_val:]
    return train, val, test

pos_tr, pos_val, pos_te = split_list(pos_files)
neg_tr, neg_val, neg_te = split_list(neg_files)

def copy_files(files, dst_dir):
    for f in files:
        dst = dst_dir/f.name
        # ป้องกันชื่อชนกัน
        stem, ext = f.stem, f.suffix
        k = 1
        while dst.exists():
            dst = dst_dir/f"{stem}_{k}{ext}"
            k += 1
        shutil.copy2(f, dst)

copy_files(pos_tr, DATASET_DIR/'train'/'crack')
copy_files(pos_val, DATASET_DIR/'val'/'crack')
copy_files(pos_te, DATASET_DIR/'test'/'crack')

copy_files(neg_tr, DATASET_DIR/'train'/'no_crack')
copy_files(neg_val, DATASET_DIR/'val'/'no_crack')
copy_files(neg_te, DATASET_DIR/'test'/'no_crack')

print("Dataset prepared at:", DATASET_DIR)
for split in ['train','val','test']:
    n_pos = len(list((DATASET_DIR/split/'crack').glob('*')))
    n_neg = len(list((DATASET_DIR/split/'no_crack').glob('*')))
    print(f"{split:5s} -> crack={n_pos}, no_crack={n_neg}")

Dataset prepared at: /content/crack_dataset_cls
train -> crack=14000, no_crack=14000
val   -> crack=3000, no_crack=3000
test  -> crack=3000, no_crack=3000


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
# import YOLO model
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s-cls.pt') # load a pretrained model (recommended for training)

# Train the model
model.train(data='/content/crack_dataset_cls/', epochs=5)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/crack_dataset_cls/, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrain

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 243.1±84.7 MB/s, size: 4.9 KB)


train: Scanning /content/crack_dataset_cls/train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:08<00:00, 3139.11it/s]


train: New cache created: /content/crack_dataset_cls/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 209.5±84.0 MB/s, size: 5.0 KB)


...

                   all      0.998          1

5 epochs completed in 0.087 hours.


Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/crack_dataset_cls/train... found 28000 images in 2 classes ✅ 
val: /content/crack_dataset_cls/val... found 6000 images in 2 classes ✅ 
test: /content/crack_dataset_cls/test... found 6000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 188/188 [00:03<00:00, 50.00it/s]


                   all      0.998          1
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fee4e239510>
curves: []
curves_results: []
fitness: 0.9991666674613953
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9983333349227905, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9991666674613953}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.07481787350025115, 'inference': 0.24890528516656713, 'loss': 0.00043239200010702916, 'postprocess': 0.0008633593333418806}
task: 'classify'
top1: 0.9983333349227905
top5: 1.0

In [ ]:
# Validate the model
metrics = model.val() # no arguments needed, dataset and settings remembered
metrics.top1 # top1 accuracy
metrics.top5 # top5 accuracy

Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/crack_dataset_cls/train... found 28000 images in 2 classes ✅ 
val: /content/crack_dataset_cls/val... found 6000 images in 2 classes ✅ 
test: /content/crack_dataset_cls/test... found 6000 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 235.3±102.2 MB/s, size: 5.0 KB)


val: Scanning /content/crack_dataset_cls/val... 6000 images, 0 corrupt: 100%|██████████| 6000/6000 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 375/375 [00:04<00:00, 92.11it/s] 


                   all      0.998          1
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


1.0

In [ ]:
!zip -r yolo8ncls_runs.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/classify/ (stored 0%)
  adding: runs/classify/train2/ (stored 0%)
  adding: runs/classify/train2/val_batch1_labels.jpg (deflated 2%)
  adding: runs/classify/train2/val_batch0_labels.jpg (deflated 1%)
  adding: runs/classify/train2/val_batch1_pred.jpg (deflated 2%)
  adding: runs/classify/train2/val_batch2_labels.jpg (deflated 2%)
  adding: runs/classify/train2/val_batch0_pred.jpg (deflated 1%)
  adding: runs/classify/train2/confusion_matrix.png (deflated 34%)
  adding: runs/classify/train2/val_batch2_pred.jpg (deflated 2%)
  adding: runs/classify/train2/confusion_matrix_normalized.png (deflated 36%)
  adding: runs/classify/train/ (stored 0%)
  adding: runs/classify/train/val_batch1_labels.jpg (deflated 1%)
  adding: runs/classify/train/train_batch1.jpg (deflated 4%)
  adding: runs/classify/train/val_batch0_labels.jpg (deflated 1%)
  adding: runs/classify/train/val_batch1_pred.jpg (deflated 1%)
  adding: runs/classify/train/val_batch2_labels.jp

In [ ]:
# Load a model
model = YOLO('yolov8s-cls.pt') # load a pretrained model (recommended for training)

# Train the model
model.train(data='/content/crack_dataset_cls/', epochs=5)

Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/crack_dataset_cls/, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

train: /content/crack_dataset_cls/train... found 28000 images in 2 classes ✅ 
val: /content/crack_dataset_cls/val... found 6000 images in 2 classes ✅ 
test: /content/crack_dataset_cls/test... found 6000 images in 2 classes ✅ 
Overriding model.yaml nc=1000 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics

train: Scanning /content/crack_dataset_cls/train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 72.5±18.6 MB/s, size: 5.0 KB)


val: Scanning /content/crack_dataset_cls/val... 6000 images, 0 corrupt: 100%|██████████| 6000/6000 [00:00<?, ?it/s]


...

                   all      0.999          1



5 epochs completed in 0.091 hours.
Optimizer stripped from runs/classify/train3/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train3/weights/best.pt, 10.3MB

Validating runs/classify/train3/weights/best.pt...
Ultralytics 8.3.181 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
train: /content/crack_dataset_cls/train... found 28000 images in 2 classes ✅ 
val: /content/crack_dataset_cls/val... found 6000 images in 2 classes ✅ 
test: /content/crack_dataset_cls/test... found 6000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 188/188 [00:03<00:00, 49.58it/s]


                   all      0.999          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train3


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fee6f25cb10>
curves: []
curves_results: []
fitness: 0.9995000064373016
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9990000128746033, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9995000064373016}
save_dir: PosixPath('runs/classify/train3')
speed: {'preprocess': 0.08837561116706638, 'inference': 0.3054126156668341, 'loss': 0.0022996573331208006, 'postprocess': 0.006077828833516226}
task: 'classify'
top1: 0.9990000128746033
top5: 1.0